# Import Libraries

In [1]:
import pandas as pd
import re
import ssl
import numpy as np
import matplotlib.pyplot as plt
from urllib.request import urlopen
from bs4 import BeautifulSoup
from konlpy.tag import Komoran
from konlpy.tag import Okt
from collections import Counter
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import ssl
from urllib.request import urlopen


torch.manual_seed(1)

# Preprocess DATA

In [3]:
tags = pd.read_csv('tag200.csv')

In [4]:
tags.columns = ['title', 'link', 'image', 'description', 'cleantxt', 'tags']
# tags.tail()

In [5]:
# check cleantxt contents

for index, row in tags.iterrows():
    # print(row.cleantxt+'\n\n\n')
    pass

In [6]:
# konlpy 사용해서 전처리
# stopwords 없애기, 어간/어미 등으로 나누기
context = ssl._create_unverified_context()
resp = urlopen('https://www.ranks.nl/stopwords/korean',context=context)
dom = BeautifulSoup(resp.read().decode('utf8'),'html.parser')
tds = dom.select('.panel-body > table > tbody > tr > td')

In [7]:
stopwords=[]
for _ in tds:
    stopwords+=re.findall(r'([ \w]+)<br/>',str(_))
    
stopwords.append('만큼')
stopwords.append('만 못하다')
stopwords.append('영')


In [8]:
# LSTM에 더 잘 적용 가능하도록 konlpy로 전처리 진행
komo = Komoran() #어미/어근의 분류가 잘 되어있다 판단.

class Tokenize:
    def __init__(self):
        pass
    def __call__(self, text):
        #stemming
        #어미 관련 모두 제외
        stemmed = [_ for _ in komo.pos(text) if _[1] not in ['EC', 'EF', 'EP', 'ETM', 'ETN']]
        
        #lemmatizing
        #VV=동사어근, VA=형용사어근
        lemma = [_[0]+'다' if (_[1]=='VA') or (_[1]=='VV') else _[0] for _ in stemmed]
        
        #remove stopwords
        tokens = [_ for _ in lemma if _ not in stopwords]
        
        return tokens
    

In [9]:
token = Tokenize()

In [10]:
a = []


for index, row in tags.iterrows():
    a.append(token(row['cleantxt']))
    
    
    
print(len(a))

200


In [11]:
tags.head()

,title,link,image,description,cleantxt,tags
0,??중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다,https://www.koreapas.com/bbs/view.php?id=circl...,[],??중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다!\n\r\n?수레바...,중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다중앙동아리 정치경제학연구...,"수레바퀴, 학회, 경영"
1,[2020 YOUNG TIGERS 신입단원 모집 D-day],https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/de478d7/resize'],[2020 YOUNG TIGERS 신입단원 모집]\n\r\n안녕하십니까 고려대학교 ...,2020 YOUNG TIGERS 신입단원 모집 Dday 2020 YOUNG TIGE...,"Young Tigers, 운동, 동아리"
2,??[KULSOM D-1] 마감 하루 전!!??????,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/6f7ae51/resize'],??지원 마감 하루 전입니다! \n\r\n??한 학기 같이 즐거운 추억 만들어요! ...,KULSOM D1 마감 하루 전!! 지원 마감 하루 전입니다! 한 학기 같이 즐...,"KULSOM, 학회, 경영"
3,[柔道部] 유도부 상시모집 ??,https://www.koreapas.com/bbs/view.php?id=circl...,"['https://img.koreapas.com/i/5b0b384/resize', ...",??고대 유도부 신입부원 상시모집??\r\n????아무나 와라 강하게 키워줄게???...,유도부 상시모집 고대 유도부 신입부원 상시모집 아무나 와라 강하게 키워줄게 고...,"유도부, 동아리, 운동"
4,[D-3]??KU-HOPE 2020년 1학기 리크루팅!??,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/e01d297/resize'],??재난에서 희망으로! 안녕하세요 고려대학교 집수리봉사동아리 KU-HOPE 입니다....,D3KUHOPE 2020년 1학기 리크루팅! 재난에서 희망으로! 안녕하세요 고려대학...,"KU-HOPE, 봉사, 동아리"


In [12]:

tags['tokens'] = a



# to_words = []
# for index, row in tags.iterrows():
#     x = row['tokens']
#     split_words = x.split(', ')
#     list_in_list = []
#     for i in split_words:
#         i = str(i)  # 다시 하나의 str으로 만들었다.
#         list_in_list.append(i)
#     to_words.append(list_in_list)
    
# tags['tokens'] = to_words

In [13]:


# tags의 tag를 각각의 string으로 만들어야 했는데, 전처리 과정에서 전체 string 형태로 잘못 만들었다.
# 다시 각각의 string으로 만드는 과정

to_words = []
for index, row in tags.iterrows():
    x = row['tags']
    x = x.replace(' ,', ', ')
    split_words = x.split(', ')
    list_in_list = []
    for i in split_words:
        i = str(i)  # 다시 하나의 str으로 만들었다.
        list_in_list.append(i)
    to_words.append(list_in_list)
    
tags['tags'] = to_words


tags.head()

,title,link,image,description,cleantxt,tags,tokens
0,??중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다,https://www.koreapas.com/bbs/view.php?id=circl...,[],??중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다!\n\r\n?수레바...,중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다중앙동아리 정치경제학연구...,"[수레바퀴, 학회, 경영]","[중앙동, 아리, 정치경제학, 연구회, 수레바퀴, 신입, 연인, 집합, 니다, 중앙..."
1,[2020 YOUNG TIGERS 신입단원 모집 D-day],https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/de478d7/resize'],[2020 YOUNG TIGERS 신입단원 모집]\n\r\n안녕하십니까 고려대학교 ...,2020 YOUNG TIGERS 신입단원 모집 Dday 2020 YOUNG TIGE...,"[Young Tigers, 운동, 동아리]","[2020, YOUNG, TIGERS, 신입, 단원, 모집, Dday, 2020, ..."
2,??[KULSOM D-1] 마감 하루 전!!??????,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/6f7ae51/resize'],??지원 마감 하루 전입니다! \n\r\n??한 학기 같이 즐거운 추억 만들어요! ...,KULSOM D1 마감 하루 전!! 지원 마감 하루 전입니다! 한 학기 같이 즐...,"[KULSOM, 학회, 경영]","[KULSOM, D1, 마감, 하루, 전, !!, 지원, 마감, 하루, 전, !, ..."
3,[柔道部] 유도부 상시모집 ??,https://www.koreapas.com/bbs/view.php?id=circl...,"['https://img.koreapas.com/i/5b0b384/resize', ...",??고대 유도부 신입부원 상시모집??\r\n????아무나 와라 강하게 키워줄게???...,유도부 상시모집 고대 유도부 신입부원 상시모집 아무나 와라 강하게 키워줄게 고...,"[유도부, 동아리, 운동]","[유도, 부, 상시, 모집, 고대, 유도, 부, 신입, 부원, 상시, 모집, 아무,..."
4,[D-3]??KU-HOPE 2020년 1학기 리크루팅!??,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/e01d297/resize'],??재난에서 희망으로! 안녕하세요 고려대학교 집수리봉사동아리 KU-HOPE 입니다....,D3KUHOPE 2020년 1학기 리크루팅! 재난에서 희망으로! 안녕하세요 고려대학...,"[KU-HOPE, 봉사, 동아리]","[D3, KUHOPE, 2020, 1, 학기, 리크, 루팅, !, 재난, 희망, !..."


# LSTM for Article Tagging

## Prepare data:

### 1. tag 정리

In [14]:
names = [] # tag 중 이름만 존재, 학회나 동아리 이름
all_tags = [] # 모든 태그
prop = [] # 태그 중 이름을 제외한 성격, 특징을 살린 태그


for index, row in tags.iterrows():
    x = row['tags'][0]
    names.append(x) if x not in names else names
    for i in row['tags']:
        all_tags.append(i) if i not in all_tags else all_tags
# 

prop = (list(set(all_tags) - set(names)))
print(prop)


['노래', '스터디', '운동', '동아리,', '동아리', '언어', '경영', '예술', '종교', '비거니즘', '학회', '대외활동', '친목', '모집', '사회', '봉사', '연합', 'IT', '음악']


In [15]:
print(len(names)) # 표본? 93개... 결과가 어떻게 나올지 궁금하군
print(len(all_tags))
print(len(prop))

# 93 + 19 = 112로 개수가 맞다.

93
112
19


In [16]:
tags['tags'] = tags['tags'].apply(lambda x: x[1:])

tags.head()

,title,link,image,description,cleantxt,tags,tokens
0,??중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다,https://www.koreapas.com/bbs/view.php?id=circl...,[],??중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다!\n\r\n?수레바...,중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다중앙동아리 정치경제학연구...,"[학회, 경영]","[중앙동, 아리, 정치경제학, 연구회, 수레바퀴, 신입, 연인, 집합, 니다, 중앙..."
1,[2020 YOUNG TIGERS 신입단원 모집 D-day],https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/de478d7/resize'],[2020 YOUNG TIGERS 신입단원 모집]\n\r\n안녕하십니까 고려대학교 ...,2020 YOUNG TIGERS 신입단원 모집 Dday 2020 YOUNG TIGE...,"[운동, 동아리]","[2020, YOUNG, TIGERS, 신입, 단원, 모집, Dday, 2020, ..."
2,??[KULSOM D-1] 마감 하루 전!!??????,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/6f7ae51/resize'],??지원 마감 하루 전입니다! \n\r\n??한 학기 같이 즐거운 추억 만들어요! ...,KULSOM D1 마감 하루 전!! 지원 마감 하루 전입니다! 한 학기 같이 즐...,"[학회, 경영]","[KULSOM, D1, 마감, 하루, 전, !!, 지원, 마감, 하루, 전, !, ..."
3,[柔道部] 유도부 상시모집 ??,https://www.koreapas.com/bbs/view.php?id=circl...,"['https://img.koreapas.com/i/5b0b384/resize', ...",??고대 유도부 신입부원 상시모집??\r\n????아무나 와라 강하게 키워줄게???...,유도부 상시모집 고대 유도부 신입부원 상시모집 아무나 와라 강하게 키워줄게 고...,"[동아리, 운동]","[유도, 부, 상시, 모집, 고대, 유도, 부, 신입, 부원, 상시, 모집, 아무,..."
4,[D-3]??KU-HOPE 2020년 1학기 리크루팅!??,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/e01d297/resize'],??재난에서 희망으로! 안녕하세요 고려대학교 집수리봉사동아리 KU-HOPE 입니다....,D3KUHOPE 2020년 1학기 리크루팅! 재난에서 희망으로! 안녕하세요 고려대학...,"[봉사, 동아리]","[D3, KUHOPE, 2020, 1, 학기, 리크, 루팅, !, 재난, 희망, !..."


### 2. Count token occurences & delete infrequent tokens

In [17]:
# count occurences of each word
# 참조한 사이트: https://towardsdatascience.com/multiclass-text-classification-using-lstm-in-pytorch-eac56baed8df

counts_tokens = Counter()
for index, row in tags.iterrows():
    counts_tokens.update(row['tokens'])
    #counts_tokens.update(token(row['cleantxt']))

In [18]:
# counts_tokens

In [19]:
# deleting infrequent words
print("num_words before: ", len(counts_tokens.keys()))
for word in list(counts_tokens):
    if counts_tokens[word] < 2:
        del counts_tokens[word]
print("num_words after:",len(counts_tokens.keys()))

num_words before:  4878
num_words after: 3480


### 3. vocab to index mapping! 

In [20]:
#creating vocabulary
vocab2index = {"":0, "UNK":1}    # adding a vector for unknown words
words = ["", "UNK"]
for word in counts_tokens:
    vocab2index[word] = len(words)
    words.append(word)
    
    
# words

### 4. tag to index mapping!

In [21]:
counts_tags = Counter()
for index, row in tags.iterrows():
    counts_tags.update(row['tags'])
    #counts_tokens.update(token(row['cleantxt']))

In [22]:
tag2index = {}
tagwords = []
for tagword in counts_tags:
    tag2index[tagword] = len(tagwords)
    tagwords.append(tagword)
    
# tagwords

### 5. encoding words

In [23]:
## find the number appropriate for N in encode_sentence function below

article_num = 0
word_num = 0

for index, row in tags.iterrows():
    print(len(row['tokens']))
    word_num += len(row['tokens'])
    article_num += 1
    

# print("the mean word number is: ", word_num/article_num)

272
142
1136
307
376
979
455
301
233
577
296
536
236
59
468
337
257
450
231
1104
369
308
352
974
216
487
230
454
595
263
178
217
306
370
470
141
539
121
1154
318
290
492
352
296
307
337
156
228
120
419
166
37
437
293
173
360
492
990
467
1205
127
231
264
199
261
727
283
460
337
252
507
546
379
977
260
273
474
352
155
307
296
469
150
232
327
702
141
492
155
173
387
378
987
1114
419
309
319
352
347
468
274
474
307
341
541
337
984
746
239
459
192
538
305
595
327
989
258
455
175
148
245
1141
284
141
989
492
579
307
295
389
468
123
449
216
337
192
248
223
724
193
985
334
405
222
162
459
123
326
296
243
468
169
115
337
1154
130
199
141
974
539
177
287
458
256
204
228
635
221
308
214
723
246
246
579
473
494
295
459
307
423
326
494
276
469
318
245
422
293
352
598
378
141
1160
286
216
148
23
777
295
308


In [24]:
def encode_sentence(text, vocab2index, N=400):
    tokenized = token(text)
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length
    # return encoded, length

In [25]:
tags['encoded'] = tags['cleantxt'].apply(lambda x: np.array(encode_sentence(x, vocab2index)))
# tags.head()


<ipython-input-25-8ea278a94635>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  tags['encoded'] = tags['cleantxt'].apply(lambda x: np.array(encode_sentence(x, vocab2index)))


In [26]:
tags['encoded'][0]

array([array([  2,   3,   4,   5,   6,   7,   8,   9,  10,   2,   3,   4,   5,
         6,   7,   8,  11,  12,   6,  13,  14,  15,  16,  17,  18,  19,
        20,  21,  22,  23,  24,  25,  26,  20,  27,  12,  28,  29,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  12,  39,  40,  41,  42,
        37,  43,  44,  38,  45,  46,  22,  47,  30,  31,  32,  14,  15,
        48,  49,  50,  51,  52,  12,  53,  54,  55,  56,  57,  40,  58,
        59,  60,  61,  62,  55,  63,  57,  64,  30,  31,  32,   6,  65,
        66,  67,  68,  69,   6,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  12,  80,  81,  82,  83,  81,  84,  85,  70,  86,  57,
        87,  88,  13,  89,  17,   7,   8,  90,  78,  79,  91,  25,  92,
        78,  93,  94,  56,  22,  95,  87,  96,  97,  27,  12,   6,  75,
        98,  57,  99, 100, 101, 102,  71, 103, 104, 105, 106, 107,  22,
       108, 109, 110,  52, 111, 112, 113, 114, 115, 116, 117,  15, 118,
        22, 119,  75, 119, 120, 121, 122, 123,  17, 124, 

## Using Dataset:

In [27]:
tags.head()

,title,link,image,description,cleantxt,tags,tokens,encoded
0,??중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다,https://www.koreapas.com/bbs/view.php?id=circl...,[],??중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다!\n\r\n?수레바...,중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다중앙동아리 정치경제학연구...,"[학회, 경영]","[중앙동, 아리, 정치경제학, 연구회, 수레바퀴, 신입, 연인, 집합, 니다, 중앙...","[[2, 3, 4, 5, 6, 7, 8, 9, 10, 2, 3, 4, 5, 6, 7..."
1,[2020 YOUNG TIGERS 신입단원 모집 D-day],https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/de478d7/resize'],[2020 YOUNG TIGERS 신입단원 모집]\n\r\n안녕하십니까 고려대학교 ...,2020 YOUNG TIGERS 신입단원 모집 Dday 2020 YOUNG TIGE...,"[운동, 동아리]","[2020, YOUNG, TIGERS, 신입, 단원, 모집, Dday, 2020, ...","[[175, 176, 177, 7, 178, 11, 179, 175, 176, 17..."
2,??[KULSOM D-1] 마감 하루 전!!??????,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/6f7ae51/resize'],??지원 마감 하루 전입니다! \n\r\n??한 학기 같이 즐거운 추억 만들어요! ...,KULSOM D1 마감 하루 전!! 지원 마감 하루 전입니다! 한 학기 같이 즐...,"[학회, 경영]","[KULSOM, D1, 마감, 하루, 전, !!, 지원, 마감, 하루, 전, !, ...","[[256, 257, 258, 259, 188, 260, 209, 258, 259,..."
3,[柔道部] 유도부 상시모집 ??,https://www.koreapas.com/bbs/view.php?id=circl...,"['https://img.koreapas.com/i/5b0b384/resize', ...",??고대 유도부 신입부원 상시모집??\r\n????아무나 와라 강하게 키워줄게???...,유도부 상시모집 고대 유도부 신입부원 상시모집 아무나 와라 강하게 키워줄게 고...,"[동아리, 운동]","[유도, 부, 상시, 모집, 고대, 유도, 부, 신입, 부원, 상시, 모집, 아무,...","[[648, 184, 649, 11, 160, 648, 184, 7, 650, 64..."
4,[D-3]??KU-HOPE 2020년 1학기 리크루팅!??,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/e01d297/resize'],??재난에서 희망으로! 안녕하세요 고려대학교 집수리봉사동아리 KU-HOPE 입니다....,D3KUHOPE 2020년 1학기 리크루팅! 재난에서 희망으로! 안녕하세요 고려대학...,"[봉사, 동아리]","[D3, KUHOPE, 2020, 1, 학기, 리크, 루팅, !, 재난, 희망, !...","[[762, 763, 175, 73, 74, 764, 765, 12, 502, 74..."


In [28]:
tags_replica = tags.copy()
data = pd.DataFrame(columns = ['title', 'link', 'image', 'description', 'cleantxt', 'tags', 'tokens', 'encoded'], index=tags.index)
num=0

for index, row in tags_replica.iterrows():
    for x in row['tags']:
        data.loc[num] = row
        data['tags'][num] =x 
        num+=1

        
data.head()

,title,link,image,description,cleantxt,tags,tokens,encoded
0,??중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다,https://www.koreapas.com/bbs/view.php?id=circl...,[],??중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다!\n\r\n?수레바...,중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다중앙동아리 정치경제학연구...,학회,"[중앙동, 아리, 정치경제학, 연구회, 수레바퀴, 신입, 연인, 집합, 니다, 중앙...","[[2, 3, 4, 5, 6, 7, 8, 9, 10, 2, 3, 4, 5, 6, 7..."
1,??중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다,https://www.koreapas.com/bbs/view.php?id=circl...,[],??중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다!\n\r\n?수레바...,중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다중앙동아리 정치경제학연구...,경영,"[중앙동, 아리, 정치경제학, 연구회, 수레바퀴, 신입, 연인, 집합, 니다, 중앙...","[[2, 3, 4, 5, 6, 7, 8, 9, 10, 2, 3, 4, 5, 6, 7..."
2,[2020 YOUNG TIGERS 신입단원 모집 D-day],https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/de478d7/resize'],[2020 YOUNG TIGERS 신입단원 모집]\n\r\n안녕하십니까 고려대학교 ...,2020 YOUNG TIGERS 신입단원 모집 Dday 2020 YOUNG TIGE...,운동,"[2020, YOUNG, TIGERS, 신입, 단원, 모집, Dday, 2020, ...","[[175, 176, 177, 7, 178, 11, 179, 175, 176, 17..."
3,[2020 YOUNG TIGERS 신입단원 모집 D-day],https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/de478d7/resize'],[2020 YOUNG TIGERS 신입단원 모집]\n\r\n안녕하십니까 고려대학교 ...,2020 YOUNG TIGERS 신입단원 모집 Dday 2020 YOUNG TIGE...,동아리,"[2020, YOUNG, TIGERS, 신입, 단원, 모집, Dday, 2020, ...","[[175, 176, 177, 7, 178, 11, 179, 175, 176, 17..."
4,??[KULSOM D-1] 마감 하루 전!!??????,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/6f7ae51/resize'],??지원 마감 하루 전입니다! \n\r\n??한 학기 같이 즐거운 추억 만들어요! ...,KULSOM D1 마감 하루 전!! 지원 마감 하루 전입니다! 한 학기 같이 즐...,학회,"[KULSOM, D1, 마감, 하루, 전, !!, 지원, 마감, 하루, 전, !, ...","[[256, 257, 258, 259, 188, 260, 209, 258, 259,..."


In [29]:
data.info()

# enlarged the dataframe so the y target could be just 1

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396 entries, 0 to 395
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        396 non-null    object
 1   link         396 non-null    object
 2   image        396 non-null    object
 3   description  396 non-null    object
 4   cleantxt     396 non-null    object
 5   tags         396 non-null    object
 6   tokens       396 non-null    object
 7   encoded      396 non-null    object
dtypes: object(8)
memory usage: 47.8+ KB


In [30]:
## tag2index = {v: k for k, v in tag2index.items()}

# key- value mapping tags so tags could be encoded to numbers
data = data.replace({"tags":tag2index})


In [31]:
tag2index

{'학회': 0,
 '경영': 1,
 '운동': 2,
 '동아리': 3,
 '봉사': 4,
 '언어': 5,
 '친목': 6,
 'IT': 7,
 '종교': 8,
 '사회': 9,
 '대외활동': 10,
 '스터디': 11,
 '예술': 12,
 '음악': 13,
 '동아리,': 14,
 '비거니즘': 15,
 '모집': 16,
 '연합': 17,
 '노래': 18}

In [32]:
data.tail()

,title,link,image,description,cleantxt,tags,tokens,encoded
391,[오늘 8시 마감]??고려대학교 유일의 브랜드 컨설팅 학회 CREAKET에서 14기...,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/3095002/resize'],????오늘 8시 서류 마감????\r\n????망설이지 말고 지원해주세요????\...,오늘 8시 마감고려대학교 유일의 브랜드 컨설팅 학회 CREAKET에서 14기를 모집...,0,"[오늘, 8, 시, 마감, 고려대학교, 유일, 브랜드, 컨설팅, 학회, CREAKE...","[[1237, 825, 32, 258, 181, 897, 1970, 421, 17,..."
392,[D-1] ?Partyproviders 27th 2차 리크루팅? ~3.22,https://www.koreapas.com/bbs/view.php?id=circl...,"['https://img.koreapas.com/i/314908f/resize', ...","많은 문의와 요청에 힘입어, Partyproviders의 2차 리크루팅이 오늘부터 ...",D1 Partyproviders 27th 2차 리크루팅 3.22 많은 문의와 요청에...,3,"[D1, Partyproviders, 27, th, 2, 차, 리크, 루팅, 3, ...","[[257, 3421, 1812, 317, 197, 1203, 764, 765, 2..."
393,[D-1] ?Partyproviders 27th 2차 리크루팅? ~3.22,https://www.koreapas.com/bbs/view.php?id=circl...,"['https://img.koreapas.com/i/314908f/resize', ...","많은 문의와 요청에 힘입어, Partyproviders의 2차 리크루팅이 오늘부터 ...",D1 Partyproviders 27th 2차 리크루팅 3.22 많은 문의와 요청에...,12,"[D1, Partyproviders, 27, th, 2, 차, 리크, 루팅, 3, ...","[[257, 3421, 1812, 317, 197, 1203, 764, 765, 2..."
394,[KUHS] 고려대학교 ‘호스피스 봉사동아리 (KUHS)’에서 신입부원을 모집합니다...,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/33967e1/resize'],?? KUHS (Korea University Hospice Society; 고려대...,KUHS 고려대학교 호스피스 봉사동아리 (KUHS)에서 신입부원을 모집합니다! (일...,4,"[KUHS, 고려대학교, 호스피스, 봉사, 동아리, (, KUHS, ), 신입, 부...","[[2470, 181, 2471, 785, 664, 16, 2470, 28, 7, ..."
395,[KUHS] 고려대학교 ‘호스피스 봉사동아리 (KUHS)’에서 신입부원을 모집합니다...,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/33967e1/resize'],?? KUHS (Korea University Hospice Society; 고려대...,KUHS 고려대학교 호스피스 봉사동아리 (KUHS)에서 신입부원을 모집합니다! (일...,3,"[KUHS, 고려대학교, 호스피스, 봉사, 동아리, (, KUHS, ), 신입, 부...","[[2470, 181, 2471, 785, 664, 16, 2470, 28, 7, ..."


In [33]:
X = list(data['encoded'])
y = list(data['tags'])
indices = data.index
from sklearn.model_selection import train_test_split
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1)


X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X, y, indices, test_size=0.1)  # training dataset
X_val, X_test, y_val, y_test, indices_val, indices_test = train_test_split(X_test, y_test, indices_test, test_size=0.5)  # validation and test dataset

In [34]:
print(len(y_train))
print(len(y_val))
print(len(y_test))

356
20
20


In [35]:
class ArticleDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

In [36]:
train_ds = ArticleDataset(X_train, y_train)
valid_ds = ArticleDataset(X_val, y_val)
test_ds = ArticleDataset(X_test, y_test)

### Create the model: 

In [37]:
def train_model(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long()
            y = y.long()
            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, val_rmse = validation_metrics(model, val_dl)
        if i % 5 == 1:
            print("train loss %.3f, val loss %.3f, val accuracy %.3f, and val rmse %.3f" % (sum_loss/total, val_loss, val_acc, val_rmse))
            # print("train loss %.3f, val loss %.3f, val accuracy %.3f, and val rmse %.3f" % (sum_loss/total, val_loss, val_acc, val_rmse))

def validation_metrics (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    for x, y, l in valid_dl:
        x = x.long()
        y = y.long()
        y_hat = model(x, l)
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]
    return sum_loss/total, correct/total, sum_rmse/total


### Train the model:

In [38]:
class LSTM_variable_input(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(0.3)
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 19)  ## 19: classification target size
        
    def forward(self, x, s):
        x = self.embeddings(x)
        x = self.dropout(x)
        x_pack = pack_padded_sequence(x, s, batch_first=True, enforce_sorted=False)
        out_pack, (ht, ct) = self.lstm(x_pack)
        out = self.linear(ht[-1])
        return out

In [39]:
batch_size = 100
vocab_size = len(words)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=batch_size)
test_dl = DataLoader(test_ds, batch_size = 1)

In [40]:
model = LSTM_variable_input(vocab_size, 50, 50)
train_model(model, epochs=30, lr=0.05)

# 시간이 오래 걸린다.

train loss 2.036, val loss 1.641, val accuracy 0.400, and val rmse 3.324
train loss 1.086, val loss 1.753, val accuracy 0.150, and val rmse 5.422
train loss 1.046, val loss 2.108, val accuracy 0.200, and val rmse 5.775
train loss 0.956, val loss 2.037, val accuracy 0.250, and val rmse 4.550
train loss 0.971, val loss 1.853, val accuracy 0.200, and val rmse 5.775
train loss 0.935, val loss 2.144, val accuracy 0.250, and val rmse 5.771


In [41]:
# test dataset output 결과
pred_list = []

for x, y, l in test_dl:
    x = x.long()
    y= y.long()
    y_hat = model(x, l)
    # print(type(pred))
    pred = torch.max(y_hat, 1)[1].data.cpu().numpy()[0]
    pred_list.append(pred)
    
pred_list

[0, 4, 3, 0, 3, 3, 12, 12, 0, 2, 0, 3, 1, 3, 4, 3, 1, 0, 6, 0]

In [44]:
test_df = data.iloc[indices_test]

In [47]:
tag2index = {v: k for k, v in tag2index.items()}

# key- value mapping tags so tags could be encoded to numbers
test_df = test_df.replace({"tags":tag2index})
test_df

,title,link,image,description,cleantxt,tags,tokens,encoded
341,[지원마감 D-2][C-ESI]고연서성 연합 E-LAND 산학협력학회 C-ESI에서...,https://www.koreapas.com/bbs/view.php?id=circl...,"['https://img.koreapas.com/i/e9e4a41/resize', ...","[C-ESI 14th Recruiting]\r\n""당신이 꿈꾸던 실전 프로젝트!""\...",지원마감 D2CESI고연서성 연합 ELAND 산학협력학회 CESI에서 14기 학회원...,학회,"[지원, 마감, D2, CESI, 고연, 서성, 연합, ELAND, 산학, 협력, ...","[[209, 258, 1463, 3130, 187, 3128, 817, 3129, ..."
280,??마감 D-4??[이공계생 모집]교육봉사동아리 KUECS 5기 모집,https://www.koreapas.com/bbs/view.php?id=circl...,"['https://img.koreapas.com/i/9e532b8/resize', ...",안녕하세요 고려대학교 공과대학 교육봉사동아리 KUECS (쿠엑스) 입니다!\n\r\...,마감 D4이공계생 모집교육봉사동아리 KUECS 5기 모집 안녕하세요 고려대학교 공과...,봉사,"[마감, D4, 이공, 계, 생, 모집, 교육, 봉사, 동아리, KUECS, 5기,...","[[258, 2768, 1077, 1078, 674, 11, 577, 785, 66..."
328,<마감 D-1> 중앙 문학동아리 「고대문학회」에서 신입회원을 모집합니다,https://www.koreapas.com/bbs/view.php?id=circl...,"['https://img.koreapas.com/i/2e1a64e/resize', ...","중앙 문학동아리 「고대문학회」에서 신입회원을 모집합니다\n\r\n안녕하세요, 고려대...",마감 D1 중앙 문학동아리 고대문학회에서 신입회원을 모집합니다 중앙 문학동아리 고대...,예술,"[마감, D1, 중앙, 문학동, 아리, 고대, 문학회, 신입, 회원, 모집, 중앙,...","[[258, 257, 898, 3386, 3, 160, 3387, 7, 964, 1..."
297,[C-ESI]고연서성 연합 E-LAND 산학협력학회 C-ESI에서 14기 학회원을...,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/106a9da/resize'],"[C-ESI 14th Recruiting]\r\n""당신이 꿈꾸던 실전 프로젝트!""\...",CESI고연서성 연합 ELAND 산학협력학회 CESI에서 14기 학회원을 모집합니...,학회,"[CESI, 고연, 서성, 연합, ELAND, 산학, 협력, 학회, CESI, 14...","[[3130, 187, 3128, 817, 3129, 376, 377, 17, 31..."
41,고려대학교 중앙동아리 풍물패 고대농악대(KUNAD),https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/b1e2dc4/resize'],??안녕하세요~? 우리 아기호랑이들!!?????? \r\n살아움직이며 실천하는 풍물...,고려대학교 중앙동아리 풍물패 고대농악대(KUNAD) 안녕하세요 우리 아기호랑이들!!...,동아리,"[고려대학교, 중앙동, 아리, 풍물, 패, 고대, 농악대, (, KUNAD, ), ...","[[181, 2, 3, 2112, 2113, 160, 2114, 16, 1, 28,..."
125,서울에 단 4개밖에 없는 댄스스포츠 동아리가 고대에 있다?!????,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/c51b377/resize'],??불아스 제38기 신입부원을 모집합니다!!\r\n??-춤을 좋아하는 사람!\r\n...,서울에 단 4개밖에 없는 댄스스포츠 동아리가 고대에 있다?! 불아스 제38기 신입부...,운동,"[서울, 단, 4, 개, 밖에, 없다, 댄스스포츠, 동아리, 고대, 있, ?, !,...","[[805, 1183, 238, 24, 1109, 86, 1767, 664, 160..."
392,[D-1] ?Partyproviders 27th 2차 리크루팅? ~3.22,https://www.koreapas.com/bbs/view.php?id=circl...,"['https://img.koreapas.com/i/314908f/resize', ...","많은 문의와 요청에 힘입어, Partyproviders의 2차 리크루팅이 오늘부터 ...",D1 Partyproviders 27th 2차 리크루팅 3.22 많은 문의와 요청에...,동아리,"[D1, Partyproviders, 27, th, 2, 차, 리크, 루팅, 3, ...","[[257, 3421, 1812, 317, 197, 1203, 764, 765, 2..."
120,[D-4]고려대학교 성우동아리 온보이싱 26기 신입부원 모집,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/8c318c2/resize'],"???""성우""동아리요??? 네! 온보이싱은 고려대학교 유일무이한?성우동아리로애니메이...",D4고려대학교 성우동아리 온보이싱 26기 신입부원 모집 성우동아리요? 네! 온보이싱...,동아리,"[D4, 고려대학교, 성우, 동아리, 온, 보이, 싱, 26, 기, 신입, 부원, ...","[[2768, 181, 1238, 664, 1239, 1240, 1241, 314,..."
0,??중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다,https://www.koreapas.com/bbs/view.php?id=circl...,[],??중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다!\n\r\n?수레바...,중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다중앙동아리 정치경제학연구...,학회,"[중앙동, 아리, 정치경제학, 연구회, 수레바퀴, 신입, 연인, 집합, 니다, 중앙...","[[2, 3, 4, 5, 6, 7, 8, 9, 10, 2, 3, 4, 5, 6, 7..."
207,[스쿼시] 고려대학교 중앙 스쿼시동아리 ??엔더스?? 신입부원 모집,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/0a15711/resize'],고려대학교 중앙 스쿼시동아리 ??엔더스??에서 신입부원을 모집합니다.\r\n학번무관...,스쿼시 고려대학교 중앙 스쿼시동아리 엔더스 신입부원 모집 고려대학교 중앙 스쿼시동아...,동아리,"[스쿼시, 고려대학교, 중앙, 스쿼시, 동아리, 엔더스, 신입, 부원, 모집, 고려...","[[1695, 181, 898, 1695, 664, 1696, 7, 650, 11,..."


In [48]:
test_df = test_df.drop(['link', 'image', 'description', 'tokens', 'encoded'], axis=1)

In [49]:
test_df['pred'] = pred_list
test_df = test_df.replace({"pred":tag2index})

test_df

,title,cleantxt,tags,pred
341,[지원마감 D-2][C-ESI]고연서성 연합 E-LAND 산학협력학회 C-ESI에서...,지원마감 D2CESI고연서성 연합 ELAND 산학협력학회 CESI에서 14기 학회원...,학회,학회
280,??마감 D-4??[이공계생 모집]교육봉사동아리 KUECS 5기 모집,마감 D4이공계생 모집교육봉사동아리 KUECS 5기 모집 안녕하세요 고려대학교 공과...,봉사,봉사
328,<마감 D-1> 중앙 문학동아리 「고대문학회」에서 신입회원을 모집합니다,마감 D1 중앙 문학동아리 고대문학회에서 신입회원을 모집합니다 중앙 문학동아리 고대...,예술,동아리
297,[C-ESI]고연서성 연합 E-LAND 산학협력학회 C-ESI에서 14기 학회원을...,CESI고연서성 연합 ELAND 산학협력학회 CESI에서 14기 학회원을 모집합니...,학회,학회
41,고려대학교 중앙동아리 풍물패 고대농악대(KUNAD),고려대학교 중앙동아리 풍물패 고대농악대(KUNAD) 안녕하세요 우리 아기호랑이들!!...,동아리,동아리
125,서울에 단 4개밖에 없는 댄스스포츠 동아리가 고대에 있다?!????,서울에 단 4개밖에 없는 댄스스포츠 동아리가 고대에 있다?! 불아스 제38기 신입부...,운동,동아리
392,[D-1] ?Partyproviders 27th 2차 리크루팅? ~3.22,D1 Partyproviders 27th 2차 리크루팅 3.22 많은 문의와 요청에...,동아리,예술
120,[D-4]고려대학교 성우동아리 온보이싱 26기 신입부원 모집,D4고려대학교 성우동아리 온보이싱 26기 신입부원 모집 성우동아리요? 네! 온보이싱...,동아리,예술
0,??중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다,중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다중앙동아리 정치경제학연구...,학회,학회
207,[스쿼시] 고려대학교 중앙 스쿼시동아리 ??엔더스?? 신입부원 모집,스쿼시 고려대학교 중앙 스쿼시동아리 엔더스 신입부원 모집 고려대학교 중앙 스쿼시동아...,동아리,운동
